# Introduction

We are curious what fraction of a library is filled with spikes?

In [1]:
import os
from glob import glob
import sys
import pandas

In [2]:
%matplotlib inline

In [3]:
ROOT=os.path.expanduser("~diane/proj/encode3-curation")
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [4]:
experiment_qc = pandas.read_csv(os.path.join(ROOT, 'compare-mad', 'experiment-mad-qc.csv'))

In [5]:
wold = glob(os.path.expanduser('~/public_html/compare-mad/wold_gene_quantifications/*_tpms.csv'))
len(wold)

95

In [6]:
gingeras = glob(os.path.expanduser('~/public_html/compare-mad/gingeras_fpkms/*_tpms.csv'))
len(gingeras)

142

In [ ]:
def read_experiment_quantifications(experiments):
    for pathname in experiments:
        path, filename = os.path.split(pathname)
        accession = filename.split('_')[0]
        data = pandas.read_csv(pathname, index_col=0, header=0)
        trna_rows = [ x[0].isdigit() for x in data.index ]
        gencode_rows = [ x.startswith('ENS') for x in data.index]
        spike_rows = [ x.startswith('gSpikein') for x in data.index]
        trna_sum = data[trna_rows].sum()
        gencode_sum = data[gencode_rows].sum()
        spike_sum = data[spike_rows].sum()
        total_sum = data.sum()
        fraction_spikes = spike_sum / total_sum
        for name in fraction_spikes.index:
            yield (accession, name, fraction_spikes[name])

In [ ]:
gingeras_sums = pandas.DataFrame(read_experiment_quantifications(gingeras), columns=['experiment', 'library', 'fraction'])
gingeras_sums.to_csv(os.path.expanduser('~/public_html/compare-mad/gingeras_fpkms/gingeras-fraction-spikes.csv'),
                 index=False)

# Gingeras Fraction Spikes

In [ ]:
gingeras_sums.fraction.plot(kind='hist')

There's so few Gingeras spikes > 0.05. We can look at all the values

In [ ]:
gingeras_sums[gingeras_sums.fraction > 0.05]

In [ ]:
wold_sums = pandas.DataFrame(read_experiment_quantifications(wold), columns=['experiment', 'library', 'fraction'])
wold_sums.to_csv(os.path.expanduser('~/public_html/compare-mad/wold_gene_quantifications/wold-fraction-spikes.csv'),
                 index=False)

# Wold Fraction Spikes

In [ ]:
wold_sums.fraction.plot(kind='hist')

In [ ]:
wold_spearman_fraction = pandas.merge(
    experiment_qc[['experiment', 'Spearman']], 
    wold_sums[['experiment', 'fraction']],
    on='experiment', how='inner')

# Wold Spearman vs fraction spikes

In [ ]:
wold_spearman_fraction[['Spearman', 'fraction']].plot('Spearman', 'fraction', kind='scatter')

# Wold MAD vs fraction spikes

In [ ]:
wold_spearman_fraction = pandas.merge(
    experiment_qc[['experiment', 'MAD']], 
    wold_sums[['experiment', 'fraction']],
    on='experiment', how='inner').plot('MAD', 'fraction', kind='scatter')

In [ ]:
wold_fraction = pandas.merge(
    experiment_qc[['experiment', 'description', 'Spearman']], 
    wold_sums[['experiment', 'library', 'fraction']],
    on='experiment', how='inner')

In [ ]:
wold_fraction[wold_fraction['fraction'] > 0.10][['experiment', 'description', 'Spearman', 'library', 'fraction']]